# Hello!

In this notebook we will be scraping the below Wikipedia web page to extract all the postal codes in Canada. Using this information we will then begin getting geographical information of each neighborhood and, with the help of Foursquare API, get venues within these neighborhoods.

Ultimately, our goal is to group venue categories into clusters that could make us understand the neighborhood dynamic.

In this notebook, we will complete clustering for Toronto, Ontario only.



<h3 id="header">PART 1</h3>


**In order to segment and cluster the neighborhoods in Toronto, we will need the below libraries installed. In order to scrape the below Wikipedia page, we will be using Beautiful Soup library.**
('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') 


In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import urllib.request


**We will be requesting the Wiki page and reading the page in a row format. In order to make better sense of the page, we will be using "prettify" function.**

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)

soup = BeautifulSoup(page, "lxml")

#print(soup.prettify())


In [3]:
table=soup.find('table',{'class':'wikitable sortable'})

<Header>

**Because we do not need the entire page, we will only scrape those with "tr"**

In [4]:
links = table.find_all('tr')
#links

**We need to begin appending the table in the URL into our data frame so we first begin with creating an empty list**

In [5]:
Data=[]

for link in links:
    Data.append([t.text.strip()for t in link.find_all('td')])




In [6]:
    
df=pd.DataFrame(Data,columns=['PostalCode','Borough','Neighborhood'])
df=df[df['Borough']!='Not assigned']
df = df[~df['PostalCode'].isnull()] 


df

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
161,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
166,M4Y,Downtown Toronto,Church and Wellesley
169,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
170,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


**More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.**

In [7]:
df2=df.groupby('PostalCode').agg(lambda x: ','.join(x))

df3=df.groupby('PostalCode')



**If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.**


In [8]:
 df['Neighborhood'].replace('Not Assigned',df['Borough'],inplace=True)
df                            


,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
161,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
166,M4Y,Downtown Toronto,Church and Wellesley
169,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
170,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [9]:
df.shape                           


(103, 3)

<h3 id="header">PART 2</h3>


**Using Geocoder, we will then extract Longitude and Latitude of each PostalCode**

In [68]:
 pip install geocoder


     |████████████████████████████████| 102kB 1.3MB/s ta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [10]:
import geocoder # import geocoder


In [15]:
post=pd.read_csv('Geospatial_Coordinates.csv')
post

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [18]:
post.rename(columns={'Postal Code':'PostalCode'}, 
                 inplace=True)
post

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [20]:
df_merge=df.merge(post, on='PostalCode', how='left')
df_merge

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<h3 id="header">PART 3</h3>


**In this part, we will be mapping Toronto and begin the clustering process based on neighborhood and venue category**

In [24]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0
  python_abi         conda-forge/osx-64::python_abi-3.7-1_cp37m

The following packages will be UPDATED:

  

In [28]:

address = 'Toronto,ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [32]:
!conda install -c conda-forge folium=0.5.0 --yes # 


Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         860 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1
  branca   

In [33]:
import folium

In [42]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [43]:
for lat, lng, borough, neighborhood in zip(
        df_merge['Latitude'], 
        df_merge['Longitude'], 
        df_merge['Borough'], 
        df_merge['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [44]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KIRCLRF5USTIPBY25REXDGR4C5MVBDKC50MLXRWTNRFOMVPG
CLIENT_SECRET:QNO15UBG3AG0FGIWGQ25ZONJ5UONI3N5XOYEZTSO0KAEZH41


<h3 id="header">PART 4</h3>

**We will now be exploring the first neighborhood in our dataframe. Then we will be getting its coordinates. Using these coordinates we will be able to identify close by venues and see what types of venues they are to begin segmentation**

In [45]:

df_merge.loc[0, 'Neighborhood']


'Parkwoods'

In [48]:
df_merge_latitude= df_merge.loc[0,'Latitude']
df_merge_longitude= df_merge.loc[0,'Longitude']

neighborhood_name=df_merge.loc[0,'Neighborhood']

print('Lat and Long values of {} are {},{}'.format(neighborhood_name, df_merge_latitude,df_merge_longitude))




Lat and Long values of Parkwoods are 43.7532586,-79.3296565


**Let us now get the top 100 venues within 500 meter radius**

In [50]:
LIMIT = 100

radius = 500

url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df_merge_latitude, 
    df_merge_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=KIRCLRF5USTIPBY25REXDGR4C5MVBDKC50MLXRWTNRFOMVPG&client_secret=QNO15UBG3AG0FGIWGQ25ZONJ5UONI3N5XOYEZTSO0KAEZH41&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [51]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee5488bc1ce500e4e5031a3'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

**We will be using Foursquare to GET the information then begin cleaning the json files and writing them into a dataframe**

In [55]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [53]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [56]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,GTA Restoration,Fireworks Store,43.753396,-79.333477
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [57]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


**There were only 3 venues that are within 500 meter radius of the neighborhood and that is clearly not enough for segmentation, so let us apply this to all the neighborhoods in Toronto**

In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**Now we can run the below code to for all of Toronto and create a new dataframe for Toronto_Neighborhoods**

In [81]:

Toronto_neighborhoods2 = getNearbyVenues(names=df_merge['Neighborhood'],
                                   latitudes=df_merge['Latitude'],
                                   longitudes=df_merge['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [82]:
print(Toronto_neighborhoods2.shape)
Toronto_neighborhoods2.head()

(2132, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,GTA Restoration,43.753396,-79.333477,Fireworks Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


**Let us check how many venues were returned for each neighborhood**

In [83]:
(Toronto_neighborhoods2.groupby('Neighborhood').count())

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
...,...,...,...,...,...,...
"Willowdale, Willowdale West",6,6,6,6,6,6
Woburn,3,3,3,3,3,3
Woodbine Heights,8,8,8,8,8,8


In [85]:
print('There are {} unique categories'.format(len(Toronto_neighborhoods2['Venue Category'].unique())))

There are 280 unique categories


In [87]:
Toronto_neighborhoods2.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,GTA Restoration,43.753396,-79.333477,Fireworks Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


<h3 id="header">PART 5</h3>

**We will now begin analyzing all the neighborhoods in Toronto. In order to begin clustering them, we will have to transform all categorical velues
into numerical values using OneHotEncoder. This way all venue categories will be presented as either 0 or 1**

In [89]:
# one hot encoding
toronto_onehot3 = pd.get_dummies(Toronto_neighborhoods2[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot3['Neighborhood'] = Toronto_neighborhoods2['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns3 = [toronto_onehot3.columns[-1]] + list(toronto_onehot3.columns[:-1])
toronto_onehot3 = toronto_onehot3[fixed_columns3]

toronto_onehot3.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
toronto_onehot3.shape

(2132, 280)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [157]:
toronto_grouped = toronto_onehot3.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
91,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
92,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0
93,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0


In [163]:
toronto_grouped11 = toronto_onehot3.groupby('Neighborhood').sum().reset_index()
(toronto_grouped11)

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bayview Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92,Woodbine Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
93,York Mills West,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [161]:
toronto_grouped = toronto_onehot3.groupby('Neighborhood').count().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
1,"Alderwood, Long Branch",9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
2,"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,20,20
3,Bayview Village,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
4,"Bedford Park, Lawrence Manor East",24,24,24,24,24,24,24,24,24,...,24,24,24,24,24,24,24,24,24,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale West",6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
91,Woburn,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
92,Woodbine Heights,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
93,York Mills West,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [95]:
toronto_grouped.shape

(95, 280)

**We now would like to get each neighborhood with top 5 common venues**

In [96]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print ("----"+hood+"----")
    temp=toronto_grouped[toronto_grouped['Neighborhood']== hood].T.reset_index()
    temp.columns=['venue','freq']
    temp=temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store   0.2
1                     Lounge   0.2
2             Breakfast Spot   0.2
3  Latin American Restaurant   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.22
1            Pharmacy  0.11
2                 Pub  0.11
3  Athletics & Sports  0.11
4        Dance Studio  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
              venue  freq
0              Bank  0.10
1       Coffee Shop  0.10
2        Restaurant  0.05
3  Sushi Restaurant  0.05
4       Supermarket  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4   Miscellaneous Shop  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0               Restaurant  0.08
1       Italian Restaurant  0.08
2           Sa

           venue  freq
0           Park  0.33
1    Swim School  0.33
2       Bus Line  0.33
3    Yoga Studio  0.00
4  Metro Station  0.00


----Leaside----
                 venue  freq
0          Coffee Shop  0.12
1  Sporting Goods Shop  0.06
2         Burger Joint  0.06
3                 Bank  0.06
4        Shopping Mall  0.06


----Little Portugal, Trinity----
                           venue  freq
0                            Bar  0.11
1               Asian Restaurant  0.07
2                     Restaurant  0.04
3  Vegetarian / Vegan Restaurant  0.04
4                    Coffee Shop  0.04


----Malvern, Rouge----
                        venue  freq
0        Fast Food Restaurant   1.0
1                 Yoga Studio   0.0
2                 Men's Store   0.0
3  Modern European Restaurant   0.0
4           Mobile Phone Shop   0.0


----Milliken, Agincourt North, Steeles East, L'Amoreaux East----
                        venue  freq
0                  Playground   0.5
1                    

**We will now put that in a data frame but first, let's write a function to sort the venues in descending order**

In [97]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Now let's create the new dataframe and display the top 10 venues for each neighborhood.**

In [99]:
num_top_venues=10

indicators=['st','nd','rd']

columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Clothing Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Dance Studio,Gym,Pharmacy,Coffee Shop,Sandwich Place,Athletics & Sports,Pub,Women's Store,Distribution Center
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Sandwich Place,Sushi Restaurant,Ice Cream Shop,Deli / Bodega,Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Fried Chicken Joint
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Diner,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,"Bedford Park, Lawrence Manor East",Restaurant,Sandwich Place,Italian Restaurant,Coffee Shop,Pizza Place,Café,Pub,Sushi Restaurant,Liquor Store,Butcher


**We will then run K Means Clustering with 5 clusters**

In [103]:
from sklearn.cluster import KMeans


In [105]:
kclusters=5

toronto_grouped_clustering=toronto_grouped.drop('Neighborhood',1)

kmeans=KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([0, 4, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [132]:
neighborhoods_venues_sorted.insert(0,'Cluster Labels',kmeans.labels_)

toronto_merged= df_merge
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


toronto_merged.head(50) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Food & Drink Shop,Park,Fireworks Store,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Grocery Store,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Restaurant,Café,Theater,Historic Site,Performing Arts Venue
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Event Space,Vietnamese Restaurant,Miscellaneous Shop,Coffee Shop,Gift Shop,Donut Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Auditorium
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Field
7,M3B,North York,Don Mills,43.745906,-79.352188,0.0,Gym,Japanese Restaurant,Coffee Shop,Sporting Goods Shop,Restaurant,Beer Store,Asian Restaurant,Bike Shop,Italian Restaurant,Athletics & Sports
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,4.0,Pizza Place,Gym / Fitness Center,Gastropub,Intersection,Bank,Café,Athletics & Sports,Fast Food Restaurant,Pet Store,Pharmacy
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0.0,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Café,Cosmetics Shop,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Bookstore,Hotel


In [140]:
toronto_merged2=toronto_merged.dropna()

toronto_merged2


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Food & Drink Shop,Park,Fireworks Store,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Grocery Store,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Restaurant,Café,Theater,Historic Site,Performing Arts Venue
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Event Space,Vietnamese Restaurant,Miscellaneous Shop,Coffee Shop,Gift Shop,Donut Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Auditorium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,1.0,Park,River,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0.0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Men's Store,Café,Pub,Yoga Studio
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,0.0,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Light Rail Station,Farmers Market,Comic Shop,Park,Pizza Place
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,0.0,Baseball Field,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dim Sum Restaurant


**In order to map the clusters, i had to convert the clusters into integers rather than floats**

In [141]:
toronto_merged2 = toronto_merged2.astype({'Cluster Labels': 'int'})

toronto_merged2.head(50)
                                        
                                


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Food & Drink Shop,Park,Fireworks Store,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Grocery Store,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Restaurant,Café,Theater,Historic Site,Performing Arts Venue
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Event Space,Vietnamese Restaurant,Miscellaneous Shop,Coffee Shop,Gift Shop,Donut Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Auditorium
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Field
7,M3B,North York,Don Mills,43.745906,-79.352188,0,Gym,Japanese Restaurant,Coffee Shop,Sporting Goods Shop,Restaurant,Beer Store,Asian Restaurant,Bike Shop,Italian Restaurant,Athletics & Sports
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,4,Pizza Place,Gym / Fitness Center,Gastropub,Intersection,Bank,Café,Athletics & Sports,Fast Food Restaurant,Pet Store,Pharmacy
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Café,Cosmetics Shop,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Bookstore,Hotel
10,M6B,North York,Glencairn,43.709577,-79.445073,4,Pizza Place,Japanese Restaurant,Asian Restaurant,Sushi Restaurant,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run


In [134]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [142]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_merged2['Latitude'], 
        toronto_merged2['Longitude'], 
        toronto_merged2['Neighborhood'], 
        toronto_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3 id="header">PART 6</h3>

**We are now able to cluster Toronto Neighborhoods based on venue category. However, we need to understand what these clusters are to help us understand more about these neighborhoods and venue locations**

Let us begin with **Cluster 0**

*Cluster 0 seems to be the most social area. It contains all the venues for socializing and going out. Hence I am going to call this cluster* ***Social Hub***. 

In [143]:
toronto_merged2.loc[toronto_merged2['Cluster Labels'] == 0, toronto_merged2.columns[[1] + list(range(5, toronto_merged2.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Grocery Store,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,Downtown Toronto,0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Restaurant,Café,Theater,Historic Site,Performing Arts Venue
3,North York,0,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Event Space,Vietnamese Restaurant,Miscellaneous Shop,Coffee Shop,Gift Shop,Donut Shop
4,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Auditorium
7,North York,0,Gym,Japanese Restaurant,Coffee Shop,Sporting Goods Shop,Restaurant,Beer Store,Asian Restaurant,Bike Shop,Italian Restaurant,Athletics & Sports
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,Seafood Restaurant,Salad Place,Steakhouse,Deli / Bodega
99,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Men's Store,Café,Pub,Yoga Studio
100,East Toronto,0,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Light Rail Station,Farmers Market,Comic Shop,Park,Pizza Place
101,Etobicoke,0,Baseball Field,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dim Sum Restaurant


**Cluster 1**

*Cluster 1 seems to contain mostly Parks/Gyms/Shopping Centers and Restaurants. Hence I am going to call this cluster* ***Parks, Recreation and Shopping Hub***. 


In [144]:
toronto_merged2.loc[toronto_merged2['Cluster Labels'] == 1, toronto_merged2.columns[[1] + list(range(5, toronto_merged2.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Food & Drink Shop,Park,Fireworks Store,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
21,York,1,Park,Pool,Women's Store,Greek Restaurant,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
35,East York,1,Park,Intersection,Convenience Store,Metro Station,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
61,Central Toronto,1,Park,Bus Line,Swim School,Fast Food Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
64,York,1,Park,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
66,North York,1,Park,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
83,Central Toronto,1,Gym,Park,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
85,Scarborough,1,Park,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
91,Downtown Toronto,1,Park,Trail,Playground,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
98,Etobicoke,1,Park,River,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


**Cluster 2**

*Cluster 2 seems to contain a lot of Cafetarias, followed by Women's Store and Drug Stores. Hence I shall call this* ***Coffee, Shopping and Dinner***. 


In [145]:
toronto_merged2.loc[toronto_merged2['Cluster Labels'] == 2, toronto_merged2.columns[[1] + list(range(5, toronto_merged2.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,2,Cafeteria,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


**Cluster 3**

*Cluster 3 seems to contain a lot Drugstore, Rental Car, Dessert Shop. Hence I shall call this* ***Drugstore and Business District***.

In [146]:
toronto_merged2.loc[toronto_merged2['Cluster Labels'] == 3, toronto_merged2.columns[[1] + list(range(5, toronto_merged2.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,3,Drugstore,Rental Car Location,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store


**Cluster 4**


*Cluster 4 seems to contain lots of fast food restaurants, some gymns, pubs. Hence, I shall call this cluster* ***Fast Food & Restaurants Hub***.

In [147]:
toronto_merged2.loc[toronto_merged2['Cluster Labels'] == 4, toronto_merged2.columns[[1] + list(range(5, toronto_merged2.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,4,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Field
8,East York,4,Pizza Place,Gym / Fitness Center,Gastropub,Intersection,Bank,Café,Athletics & Sports,Fast Food Restaurant,Pet Store,Pharmacy
10,North York,4,Pizza Place,Japanese Restaurant,Asian Restaurant,Sushi Restaurant,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
50,North York,4,Pizza Place,Furniture / Home Store,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
63,York,4,Pizza Place,Caribbean Restaurant,Convenience Store,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
70,Etobicoke,4,Pizza Place,Sandwich Place,Intersection,Middle Eastern Restaurant,Coffee Shop,Discount Store,Chinese Restaurant,Women's Store,Dog Run,Distribution Center
72,North York,4,Butcher,Grocery Store,Pharmacy,Pizza Place,Bank,Coffee Shop,Donut Shop,Diner,Discount Store,Distribution Center
77,Etobicoke,4,Sandwich Place,Pizza Place,Bus Line,Mobile Phone Shop,Women's Store,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
82,Scarborough,4,Pizza Place,Rental Car Location,Fast Food Restaurant,Fried Chicken Joint,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Gas Station,Pharmacy,Noodle House
89,Etobicoke,4,Grocery Store,Pizza Place,Fast Food Restaurant,Pharmacy,Beer Store,Sandwich Place,Fried Chicken Joint,Dog Run,Dim Sum Restaurant,Diner


## We are now able to cluster all the venues in Toronto in to 5 clusters

**Social Hub,
Parks, Recreation and Shopping Hub,
Coffee, Shopping and Dinner,
Drugstore and Business District,
Fast Food & Restaurants Hub**
